In [1]:
#IMPORT PACKAGES
from rdflib import Graph
import rdflib
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy
import json
from scipy.spatial.transform import Rotation   
import copy
import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
from scipy.linalg import orthogonal_procrustes
import matplotlib.pyplot as plt
import geomapi.tools.progresstools as pt

from sklearn.cluster import DBSCAN


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## INPUTS

In [4]:
name='rooms'

path=Path(os.getcwd()).parents[2]/'data'
pcd_input_path=os.path.join(path,f'{name}_labels.laz')
class_file=path/'_classes.json'

# name=name.split('_')[0]
# json_output_path=os.path.join(path,f'{name}_walls.json') 
# geometry_output_path= os.path.join(path,f'{name}_walls.obj') # these are the bounding surfaces of the reference levels (optional)

graphPath=str(path/f'{name}Graph.ttl')

grid_resolution = 0.01

Import Classes

In [5]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'Unassigned', 'id': 255, 'temp_id': -1, 'color': '#9da2ab'}, {'name': 'Floors', 'id': 0, 'temp_id': 0, 'color': '#03c2fc'}, {'name': 'Ceilings', 'id': 1, 'temp_id': 1, 'color': '#e81416'}, {'name': 'Walls', 'id': 2, 'temp_id': 2, 'color': '#ffa500'}, {'name': 'Columns', 'id': 3, 'temp_id': 3, 'color': '#faeb36'}, {'name': 'Doors', 'id': 4, 'temp_id': 4, 'color': '#79c314'}, {'name': 'Windows', 'id': 5, 'temp_id': 5, 'color': '#4b369d'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


Import Graph

In [6]:
graph=Graph().parse(graphPath)
nodes=tl.graph_to_nodes(graph)
wallBIMNodes=[n for n in nodes if 'Walls' in n.subject and type(n)==BIMNode]
wallPCDNodes=[n for n in nodes if 'Walls' in n.subject and type(n)==PointCloudNode]
ceilingsNodes=[n for n in nodes if 'Ceilings' in n.subject and type(n)==PointCloudNode]
floorsNodes=[n for n in nodes if 'Floors' in n.subject and type(n)==PointCloudNode]
clutterNodes=[n for n in nodes if 'Clutter' in n.subject and type(n)==PointCloudNode]
print(f'{len(wallBIMNodes)} wallNodes detected!')
print(f'{len(wallPCDNodes)} wallNodes detected!')
print(f'{len(ceilingsNodes)} ceilingsNodes detected!')
print(f'{len(floorsNodes)} floorsNodes detected!')
print(f'{len(clutterNodes)} clutterNodes detected!')

32 wallNodes detected!
32 wallNodes detected!
10 ceilingsNodes detected!
11 floorsNodes detected!
160 clutterNodes detected!


In [7]:
print({key:value for key, value in wallBIMNodes[1].__dict__.items() if not key.startswith('__') and not callable(key)})

{'_ifcPath': None, '_globalId': None, '_cartesianBounds': array([-13.48931333,  -9.36400216, -17.70804689, -17.00778565,
         2.43      ,   4.85      ]), '_orientedBounds': array([[-13.46424761, -17.70804689,   4.85      ],
       [ -9.36400216, -17.21714738,   4.85      ],
       [-13.46424761, -17.70804689,   2.43      ],
       [-13.48931333, -17.49868516,   4.85      ],
       [ -9.38906787, -17.00778565,   2.43      ],
       [-13.48931333, -17.49868516,   2.43      ],
       [ -9.38906787, -17.00778565,   4.85      ],
       [ -9.36400216, -17.21714738,   2.43      ]]), '_orientedBoundingBox': None, '_subject': rdflib.term.URIRef('file:///2_Walls_182_BIM'), '_graph': <Graph identifier=Nfd0c73aa3461485aab7c8bec2ee4651a (<class 'rdflib.graph.Graph'>)>, '_graphPath': None, '_path': None, '_name': None, '_timestamp': None, '_resource': None, '_cartesianTransform': array([[  1.        ,   0.        ,   0.        , -11.42665774],
       [  0.        ,   1.        ,   0.        , -1

Import PCD

In [8]:
laz=laspy.read(pcd_input_path)

In [9]:
def get_heigth(pcd):
    # Convert Open3D point cloud to NumPy array
    points = np.asarray(pcd.points)

    # Extract z-values from the array
    z_values = points[:, 2]

    # Compute the maximum z-value
    max_z_value = np.max(z_values)
    min_z_value = np.min(z_values)

    heigth = max_z_value - min_z_value

    return heigth

match point clouds with graph

In [10]:
for n in clutterNodes:#+ceilingsNodes+floorsNodes: # this is quite slow because you iterate through 2 scalar fields every time
    idx=np.where((laz['classes']==n.class_id) & (laz['objects']==n.object_id))
    pcd=o3d.geometry.PointCloud()
    pcd.points=o3d.utility.Vector3dVector(laz.xyz[idx])
    n.resource=pcd
    n.get_oriented_bounding_box()
    n.orientedBoundingBox.color=[1,0,0]

In [11]:
for n in wallPCDNodes:#+ceilingsNodes+floorsNodes: # this is quite slow because you iterate through 2 scalar fields every time
    idx=np.where((laz['classes']==n.class_id) & (laz['objects']==n.object_id))
    pcd=o3d.geometry.PointCloud()
    pcd.points=o3d.utility.Vector3dVector(laz.xyz[idx])
    n.resource=pcd
    n.get_oriented_bounding_box()
    n.orientedBoundingBox.color=[1,0,0] 

In [12]:
# joined_pcd=gmu.join_geometries([ n.resource for n in scene if n.resource is not None])
# o3d.visualization.draw_geometries([joined_pcd])

match PointCloudNodes to BIMNodes

In [13]:
for n in wallBIMNodes:
    n.derivedFrom = next(p for p in wallPCDNodes if p.subject.toPython() in [w.derivedFrom for w in wallBIMNodes])

In [14]:
joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(ut.literal_to_array(n.color)) for n in wallPCDNodes if n.resource is not None])
o3d.visualization.draw_geometries([joined_pcd])

Import Reference Levels

In [15]:
levelNodes=[n for n in nodes if 'level' in n.subject]
referenceNodes=[]
for l in levelNodes:
    new_graph=ut.get_subject_graph(graph,levelNodes[0].subject)
    n=SessionNode(graph=new_graph)
    n.get_oriented_bounding_box()
    n.resource=o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(n.orientedBoundingBox)
    referenceNodes.append(n) # something is wrong in the tl.graph_to_nodes function
levelNodes=referenceNodes
print(f'{len(levelNodes)} levelNodes detected!')

1 levelNodes detected!


Import ceilings and floors

In [16]:
print(levelNodes[0].linkedNodes)

[]


In [17]:
for n in ceilingsNodes+floorsNodes: # this is quite slow because you iterate through 2 scalar fields every time
    idx=np.where((laz['classes']==n.class_id) & (laz['objects']==n.object_id))
    pcd=o3d.geometry.PointCloud()
    pcd.points=o3d.utility.Vector3dVector(laz.xyz[idx])
    n.resource=pcd
    n.get_oriented_bounding_box()
    n.orientedBoundingBox.color=[1,1,0]

## PROCESSING

Convert dtrings from the graph to arrays

In [18]:
for n in wallBIMNodes:
    n.startpoint = np.asarray(n.startpoint[1:-1].split(), dtype=float)
    n.endpoint = np.asarray(n.endpoint[1:-1].split(), dtype=float)
    n.normal = np.asarray(n.normal[1:-1].split(), dtype=float)
    n.height = float(n.height)   

0.12
0.21085688161842892
0.16255729285924658
0.21216154092316852
0.12
0.1503236276740178
0.12
0.177998297731827
0.12
0.12
0.12
0.16107826955694107
0.12
0.12
0.12
0.12
0.16887373866577726
0.12
0.12
0.15015644067342235
0.12
0.18844006043444064
0.20418883128371332
0.163823941550619
0.1492665596385919
0.2395625511158263
0.12
0.15291447263228905
0.13440704968331868
0.14098275970841567
0.14244971082552693
0.13778056519576665


In [ ]:
def points_on_line(point1, point2, step_size):
    """
    Generate points on a line between two given points with a specified step size.

    Parameters:
    - point1: The starting point of the line.
    - point2: The ending point of the line.
    - step_size: The step size between consecutive points.

    Returns:
    - points: A list of points on the line.
    """
    # Calculate the direction vector
    direction = point2 - point1

    # Calculate the length of the line segment
    length = np.linalg.norm(direction)

    # Normalize the direction vector
    direction /= length

    # Calculate the number of steps needed
    num_steps = int(length / step_size)

    # Generate points along the line
    points = np.array([point1 + i * step_size * direction for i in range(num_steps + 1)])

    return points


In [ ]:
scene = o3d.t.geometry.RaycastingScene()

meshes = []

for n in wallBIMNodes:
    octree=pt.pcd_to_octree(n.derivedFrom.resource,7) #if octree is None else octree
    mesh=gmu.octree_to_voxelmesh(octree) #if mesh is None else mesh

    # # Create raycasting scene
    reference=o3d.t.geometry.TriangleMesh.from_legacy(mesh)
    # Add mesh to the scene
    scene.add_triangles(reference)
    meshes.append(mesh)

o3d.visualization.draw_geometries(meshes)

In [ ]:
o3d.visualization.draw_geometries(meshes)

In [ ]:
eps = 0.2  # Distance threshold for DBSCAN
min_samples = 1200  # Minimum number of points in a cluster

t_door_level = 0.5 #Offest tov level base contraint
t_min_door_width = 0.8
t_max_door_width = 3
t_min_door_height = 1.6
t_max_door_height = 2.5
t_min_door_opp = 1.8

t_window_level = 2
t_min_window_width = 0.5
t_max_window_width = 10
t_min_window_height = 0.5
t_max_window_height = 10
t_min_window_opp = 1


t_min_opening_width = 0.5
t_min_opening_height = 0.5
t_points = 2000

pcds = []

for n in wallBIMNodes:
    
    length = np.sqrt(np.sum((n.endpoint - n.startpoint)**2))
    surface = length * n.height
    image_size = (int(length / grid_resolution)+1, int(n.height / grid_resolution))

    if not surface < 3 and n.height > 1.5 and 9 < length < 10:

        min_z = np.min([n.endpoint[2], n.startpoint[2]])
        max_z = min_z + n.height
        num_z_steps = int(n.height / grid_resolution)
        z_grid = np.linspace(min_z, max_z, num_z_steps)  # Adjust the number of grid points as needed
        xyz_grid = []
        for z in z_grid:
            start = n.startpoint.copy()
            end =  n.endpoint.copy()
            start[2] = z
            end[2] = z
            xyz_grid.append(points_on_line(start, end, grid_resolution))

        grid_center = np.asarray(xyz_grid).reshape((-1, 3), order='C') 

        # Create Open3D point cloud
        # grid_center_pcd = o3d.geometry.PointCloud()
        # grid_center_pcd.points = o3d.utility.Vector3dVector(np.asarray(grid_center))
        # grid_center_pcd.paint_uniform_color([1,0,0])

        #In face is the dominant side of the wall
        grid_in = grid_center + n.normal*2
        # grid_in_pcd = o3d.geometry.PointCloud()
        # grid_in_pcd.points = o3d.utility.Vector3dVector(np.asarray(grid_in))
        # grid_in_pcd.paint_uniform_color([0,1,0])

        #out face is the other side of the dominant side
        grid_out = grid_center - n.normal*2
        # grid_out_pcd = o3d.geometry.PointCloud()
        # grid_out_pcd.points = o3d.utility.Vector3dVector(np.asarray(grid_out))
        # grid_out_pcd.paint_uniform_color([0,1,0])

        #create rays for the in side (towards the dominant side
        # Calculate the corresponding values using the given formula
        ori_x = n.normal[0] * np.ones(len(grid_center))
        ori_y = n.normal[1] * np.ones(len(grid_center))
        ori_z = n.normal[2] * np.ones(len(grid_center))
        
        pos_x = grid_in[:,0]
        pos_y = grid_in[:,1]
        pos_z = grid_in[:,2]

        # Stack the calculated values along the third axis to create the grid
        rays_in_values = np.stack((pos_x, pos_y, pos_z, -ori_x, -ori_y, -ori_z), axis=1)
        rays_in_tensor = o3d.core.Tensor(rays_in_values, dtype=o3d.core.Dtype.Float32)

        pos_x = grid_out[:,0]
        pos_y = grid_out[:,1]
        pos_z = grid_out[:,2]

        rays_out_values = np.stack((pos_x, pos_y, pos_z, ori_x, ori_y, ori_z), axis=1)
        rays_out_tensor = o3d.core.Tensor(rays_out_values, dtype=o3d.core.Dtype.Float32)

        scene = o3d.t.geometry.RaycastingScene()

        suroundings = geomapi.tools.select_nodes_with_intersecting_bounding_box(n, clutterNodes, u = 0.5, v =0.5, w =0.5)
        env = suroundings + [n]
        joined_pcd=gmu.join_geometries([ s.resource for s in env if s.resource is not None] + [n.derivedFrom.resource])

        octree=pt.pcd_to_octree(joined_pcd,7) #if octree is None else octree
        mesh=gmu.octree_to_voxelmesh(octree) #if mesh is None else mesh

        # # Create raycasting scene
        reference=o3d.t.geometry.TriangleMesh.from_legacy(mesh)
        # Add mesh to the scene
        scene.add_triangles(reference)

        
        ans_in = scene.cast_rays(rays_in_tensor)
        ans_out = scene.cast_rays(rays_out_tensor)

        hits_in = ans_in['t_hit'].numpy()
        hits_out = ans_out['t_hit'].numpy()

        # colors = np.zeros((len(hits_in), 3))
        opening_points = []
        opening_colors = []
        
        if hits_in[i] <4 and hits_out[i] < 4:
            # colors[i] = [0.5,0.5,0.5]
            thickness = 4-hits_out[i]-hits_in[i]
            if not thickness < 0.01 or not thickness > n.wallThickness*1.5: # Single faced surface
                # colors[i] = [1,0.5,0]
                if thickness < n.wallThickness: #Probable opening with a closed door
                    # colors[i] = [0,1,0]
                    opening_points.append(grid_center[i])
                    opening_colors.append([0,1,0])
                else:
                    opening_points.append(grid_center[i])
                    opening_colors.append([0,1,0])

            # elif hits_in[i] <4 or hits_out[i] < 4:
            #     colors[i] = [1,0.6,0]
            #     opening_points.append(grid_center[i])
            #     opening_colors.append([0,1,0])
            # else:
            #     colors[i] = [0,1,0]
            #     hole_points.append(grid_center[i])
    
        # grid_center_pcd.colors =o3d.utility.Vector3dVector(np.asarray(colors))
        opening_pcd = o3d.geometry.PointCloud()
        opening_pcd.points = o3d.utility.Vector3dVector(np.asarray(opening_points))
        opening_pcd.colors = o3d.utility.Vector3dVector(np.asarray(opening_colors))

        # o3d.visualization.draw_geometries([grid_center_pcd])
        # o3d.visualization.draw_geometries([hole_pcd])
        # hole_pcd.paint_uniform_color([0.5,0.5,0.5])
        
        new_pcd, ind = opening_pcd.remove_radius_outlier(nb_points = 100, radius = 0.1)
        
        # # Assuming you have the color array and the index list 'ind'
        # colors = np.asarray(hole_pcd.colors)

        # # Create a mask for points in 'ind'
        # mask = np.isin(np.arange(len(colors)), ind)

        # # Create an array of default colors ([0, 1, 0])
        # default_color = np.array([1, 0, 0])

        # # Create an array of colors for points in 'ind' ([1, 0, 0])
        # color_for_ind = np.array([0, 1, 0])

        # # Use np.where to efficiently assign colors based on the mask
        # colors = np.where(mask[:, np.newaxis], color_for_ind, default_color)

        # hole_pcd.colors =o3d.utility.Vector3dVector(np.asarray(colors))

        # # o3d.visualization.draw_geometries([hole_pcd])
        points = np.asarray(new_pcd.points)
        if len(new_pcd.points) > min_samples:

            # Perform clustering using DBSCAN
            
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            labels = dbscan.fit_predict(points)

            # # Number of clusters, ignoring noise if present (-1 label)
            # n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
            # print(f"Number of clusters: {n_clusters}")

            # # Assign colors to points based on cluster labels
            # colors = np.zeros_like(points)
            # for i, label in enumerate(labels):
            #     if label == -1:  # Noise points
            #         colors[i] = [0.5, 0.5, 0.5]  # Gray color for noise
            #     else:
            #         colors[i] = plt.cm.tab20(label % 20)[:3]  # Use a colormap for distinct cluster colors

            # # Create a new colored point cloud
            # colored_pcd = o3d.geometry.PointCloud()
            # colored_pcd.points = o3d.utility.Vector3dVector(points)
            # colored_pcd.colors = o3d.utility.Vector3dVector(colors)
            # colored_pcds.append(colored_pcd)

            # Extract unique cluster labels (excluding noise label -1)
            unique_labels = np.unique(labels[labels != -1])

            # Iterate over each cluster label and save corresponding points to a separate point cloud
            for label in unique_labels:
                cluster_points = points[labels == label]
                cluster_pcd = o3d.geometry.PointCloud()
                cluster_pcd.points = o3d.utility.Vector3dVector(cluster_points)

                n.base_constraint = next(l for l in levelNodes if l.subject.toPython() in [w.base_constraint for w in wallBIMNodes])

                # Extract the z-coordinates
                points2 = np.asarray(cluster_pcd.points)
                if not len(points2) <5000:
                    z_values = points2[:, 2]
                    unique_z_values = np.unique(z_values)

                    #Compute the width of the door
                    max_width = 0.0
                    min_width = 0.0

                    for z_value in unique_z_values:
                        # Get points with the current z-value
                        points_with_same_z = points2[z_values == z_value]

                        # Find the outermost points based on XY coordinates
                        min_x = np.min(points_with_same_z[:, 0])
                        max_x = np.max(points_with_same_z[:, 0])
                        min_y = np.min(points_with_same_z[:, 1])
                        max_y = np.max(points_with_same_z[:, 1])

                        # Compute the diagonal length of the bounding box
                        diagonal_length = np.linalg.norm([max_x - min_x, max_y - min_y])

                        if diagonal_length > max_width:
                            max_width = diagonal_length
                            
                    print("Opening Width:", max_width)

                    #Compute the Height of the door
                    lowest_z = np.min(z_values)
                    highest_z = np.max(z_values)
                    max_height = highest_z - lowest_z            

                    opening_surface = max_height * max_width

                    if n.base_constraint.height-t_door_level < lowest_z < n.base_constraint.height+t_door_level and t_min_door_width < max_width < t_max_door_width and t_min_door_height < max_height < t_max_door_height and opening_surface>t_min_door_opp:
                        pcd.paint_uniform_color([0,1,0])
                        pcds.append([pcd, n])
                        # print("door")
                    elif n.base_constraint.height-t_window_level < lowest_z < n.base_constraint.height+t_window_level and t_min_window_width < max_width < t_max_window_width and t_min_window_height < max_height < t_max_window_height and opening_surface>t_min_window_opp:
                        pcd.paint_uniform_color([0,0,1])
                        pcds.append([pcd, n])
                        # print("window")
                    elif t_min_opening_width < max_width and t_min_opening_height < max_height :
                        pcd.paint_uniform_color([0.5,0.5,0.5])
                        # print("Opening")
                        pcds.append([pcd, n])
                #     else: 
                #         print("Clutter")
                #         # pcd.paint_uniform_color([1,0,0])
                # else: 
                #         print("Clutter")
                #         # pcd.paint_uniform_color([1,0,0])

                # pcds.append([cluster_pcd, n])

            # pcds.append([new_pcd, n])


    

# o3d.visualization.draw_geometries([pcd[0] for pcd in pcds])

In [ ]:
o3d.visualization.draw_geometries([pcd[0] for pcd in pcds])

In [ ]:
# clusterd_pcds =[]
# for potential_door in pcds:
#     # Load your point cloud
#     pcd = potential_door[0]
#     eps = 0.2  # Distance threshold for DBSCAN
#     min_samples = 1200  # Minimum number of points in a cluster

#     # Convert point cloud to numpy array
#     points = np.asarray(pcd.points)
#     if len(pcd.points) > min_samples:

#         # Perform clustering using DBSCAN
        
#         dbscan = DBSCAN(eps=eps, min_samples=min_samples)
#         labels = dbscan.fit_predict(points)

#         # # Number of clusters, ignoring noise if present (-1 label)
#         # n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
#         # print(f"Number of clusters: {n_clusters}")

#         # # Assign colors to points based on cluster labels
#         # colors = np.zeros_like(points)
#         # for i, label in enumerate(labels):
#         #     if label == -1:  # Noise points
#         #         colors[i] = [0.5, 0.5, 0.5]  # Gray color for noise
#         #     else:
#         #         colors[i] = plt.cm.tab20(label % 20)[:3]  # Use a colormap for distinct cluster colors

#         # # Create a new colored point cloud
#         # colored_pcd = o3d.geometry.PointCloud()
#         # colored_pcd.points = o3d.utility.Vector3dVector(points)
#         # colored_pcd.colors = o3d.utility.Vector3dVector(colors)
#         # colored_pcds.append(colored_pcd)

#         # Extract unique cluster labels (excluding noise label -1)
#         unique_labels = np.unique(labels[labels != -1])

#         # Iterate over each cluster label and save corresponding points to a separate point cloud
#         for label in unique_labels:
#             cluster_points = points[labels == label]
#             cluster_pcd = o3d.geometry.PointCloud()
#             cluster_pcd.points = o3d.utility.Vector3dVector(cluster_points)

#             clusterd_pcds.append([cluster_pcd, potential_door[1]])

#     # Visualize the colored point cloud
#     # o3d.visualization.draw_geometries([pcd[0] for pcd in clusterd_pcds])


In [ ]:
# o3d.visualization.draw_geometries([pcd[0] for pcd in clusterd_pcds])

In [ ]:
from itertools import combinations




potential_doors =[]
for potential_door in pcds: 
    pcd = potential_door[0]
    n = potential_door[1]
    n.base_constraint = next(l for l in levelNodes if l.subject.toPython() in [w.base_constraint for w in wallBIMNodes])

    # Extract the z-coordinates
    points = np.asarray(pcd.points)
    if not len(points) <5000:
        z_values = points[:, 2]
        unique_z_values = np.unique(z_values)

        #Compute the width of the door
        max_width = 0.0
        min_width = 0.0

        for z_value in unique_z_values:
            # Get points with the current z-value
            points_with_same_z = points[z_values == z_value]

            # Find the outermost points based on XY coordinates
            min_x = np.min(points_with_same_z[:, 0])
            max_x = np.max(points_with_same_z[:, 0])
            min_y = np.min(points_with_same_z[:, 1])
            max_y = np.max(points_with_same_z[:, 1])

            # Compute the diagonal length of the bounding box
            diagonal_length = np.linalg.norm([max_x - min_x, max_y - min_y])

            if diagonal_length > max_width:
                max_width = diagonal_length
                
        print("Opening Width:", max_width)

        #Compute the Height of the door
        lowest_z = np.min(z_values)
        highest_z = np.max(z_values)
        max_height = highest_z - lowest_z            

        opening_surface = max_height * max_width

        if n.base_constraint.height-t_door_level < lowest_z < n.base_constraint.height+t_door_level and t_min_door_width < max_width < t_max_door_width and t_min_door_height < max_height < t_max_door_height and opening_surface>t_min_door_opp:
            pcd.paint_uniform_color([0,1,0])
            potential_doors.append(pcd)
            print("door")
        elif n.base_constraint.height-t_window_level < lowest_z < n.base_constraint.height+t_window_level and t_min_window_width < max_width < t_max_window_width and t_min_window_height < max_height < t_max_window_height and opening_surface>t_min_window_opp:
            pcd.paint_uniform_color([0,0,1])
            potential_doors.append(pcd)
            print("window")
        elif t_min_opening_width < max_width and t_min_opening_height < max_height :
            pcd.paint_uniform_color([0.5,0.5,0.5])
            print("Opening")
            potential_doors.append(pcd)
        else: 
            print("Clutter")
            pcd.paint_uniform_color([1,0,0])
            potential_doors.append(pcd)
    else: 
            print("Clutter")
            pcd.paint_uniform_color([1,0,0])
            potential_doors.append(pcd)

o3d.visualization.draw_geometries(potential_doors)



In [ ]:
print(potential_doors)
joined_pcd=gmu.join_geometries([n for n in potential_doors if n is not None])
o3d.visualization.draw_geometries([joined_pcd])



In [ ]:
for n in 